## Import relevant packages ##

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
from astropy.io import ascii
import corner
import os
from timeit import default_timer as timer
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.lines as mlines
from scipy.optimize import least_squares, curve_fit
from scipy.stats import f
from astropy.table import Table, vstack
import pandas as pd
import lmfit

## Define plotting functions:

In [4]:
def plot_data(ax, sm, data, mode, scaled=False, **kwargs):
    telescope_marker_dict = {'VLA':'s', 'ALMA':'o', 'e-MERLIN':'d'}

    for row in data:
        if mode == 'lc':
            x = row['time']
            # set marker color based on frequency
            freq = row['freq']
            colorval = sm.to_rgba(freq)
        if mode == 'sed':
            x = row['freq']
            # set marker color based on time
            time = row['time']
            colorval = sm.to_rgba(time)

        telescope = row['telescope']
        marker = telescope_marker_dict[telescope]
        
        if scaled:
            flux = row['scaled_flux']
            err = row['scaled_flux_err']
        else:
            flux = row['flux']
            err = row['flux_err']

        ax.errorbar(x, flux, yerr=err, marker=marker, c=colorval)
    return

def make_plot(data, mode, title='', xlabel='', ylabel='', cbar=True, scaled=False, model=None, params=None, model_name=None, plot_models=False):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)

    if cbar:
        # get the scalar map, plot the data using the plot_data function
        sm = cmap_setup(mode)
        plot_data(ax, sm, data, mode, scaled=scaled)

        # set up colorbar
        if mode == 'lc':
            fig.colorbar(sm, fraction=0.046, label=r'$\nu$ [GHz]')
        elif mode == 'sed':
            fig.colorbar(sm, fraction=0.046, label='time [Days]')
    else:
        sm = None
        plot_data(ax, sm, data, mode, scaled=scaled)

    # set axis scales to log
    ax.set_yscale('log')
    ax.set_xscale('log')

    #Label axes, set axis limits etc.
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    if scaled:
        ax.set_ylabel('Scaled Flux Density (mJy)')
        ax.set_title('Scaled to 3 GHz')
    else:
        ax.set_ylabel(ylabel)

    if mode == 'lc':
        x = data['time']
    elif mode == 'sed':
        x = data['freq']

    if model!=None:
        plot_model(model, params, x, ax, model_name)


def cmap_setup(mode, cmap='viridis', min_freq=0, max_freq=300, min_time=0, max_time=2000):
    '''
    color markers by frequency/time
    '''
    if mode == 'lc':
        freq_cmap = plt.cm.get_cmap(cmap)
        
        cNorm  = colors.Normalize(vmin=min_freq, vmax=max_freq)
        scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
        sm = scalarMap
        sm._A = []
    elif mode == 'sed':
        time_cmap = plt.cm.get_cmap(cmap)
        
        cNorm  = colors.Normalize(vmin=min_time, vmax=max_time)
        scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
        sm = scalarMap
        sm._A = []
    
    return sm   


# define model plotting function to be incorporated into makeplot()
def plot_model(model, params, x, ax, label):
    fit = model(x, *params)
    ax.plot(x, fit, label=label)
    ax.legend()
    return

## Input Flux Density Values

In [63]:
# create another data table with the data from my project:
# observation type:
# 6211, 6355, 6488, 5859, 6035, 7037, 6759, 7214
observation = np.array(['KuD', 'XD', 'CD', 'KD', 'KaD', 'KA', 'XA', 'KaA'])

# days since explosion
phase = np.array([1363.2, 1365.1, 1368.1, 1369.2, 1369.1, 1688.1, 1689.9, 1702.2])

# frequency of each observation (GHz)
freq = np.array([15.1, 10.0, 6.0, 22.0, 33.0, 22.0, 10.0, 33.0])

# fluxes at each time/frequency (mJy)
flux = np.array([6.476, 7.023, 8, 7.055, 8, 3.485, 3, 3.072])

# 1 sigma flux errors at each time/frequency (mJy)
flux_err = np.array([0.818, 0.944, 0.5, 0.673, 0.027, 0.468, 0.5, 0.177])

# telescope observation was made with
telescope = np.array(['VLA']*8)

table = np.column_stack((observation, phase, freq, flux, flux_err, telescope))
data = Table(table, names=['observation', 'time', 'freq', 'flux', 'flux_err', 'telescope'])
data['time'] = data['time'].astype(float)
data['freq'] = data['freq'].astype(float)
data['flux'] = data['flux'].astype(float)
data['flux_err'] = data['flux_err'].astype(float)
data

observation,time,freq,flux,flux_err,telescope
str32,float64,float64,float64,float64,str32
KuD,1363.2,15.1,6.476,0.818,VLA
XD,1365.1,10.0,7.023,0.944,VLA
CD,1368.1,6.0,8.0,0.5,VLA
KD,1369.2,22.0,7.055,0.673,VLA
KaD,1369.1,33.0,8.0,0.027,VLA
KA,1688.1,22.0,3.485,0.468,VLA
XA,1689.9,10.0,3.0,0.5,VLA
KaA,1702.2,33.0,3.072,0.177,VLA


## Define the relations that will be used for fitting:

SSA dominant case:
$$\tau_{\nu}^{\text{SSA}} = K_2\nu^{-(p+4)/2}$$
$$F_{\nu} = K_1\nu^{5/2}(1-e^{-\tau_{\nu}^{\text{SSA}}})$$

FFA dominant case:
$$\tau_{\nu}^{\text{FFA}} = K_2\nu^{-2.1}$$
$$F_{\nu} = K_1\nu^{-\alpha}e^{-\tau_{\nu}^{\text{FFA}}}$$

In [59]:
def F_SSA(freq, K1, K2, p):
    """
    Calculate the flux density using Synchotron Self Absorption (SSA) model.

    Parameters:
    freq (float or numpy.ndarray): Frequency values.
    K1 (float): Scaling factor.
    K2 (float): Scaling factor for the optical depth.
    p (float): Power-law index.

    Returns:
    F (numpy.ndarray): Flux density values.
    """
    tau = K2 * (freq)**(-(p + 4) / 2)
    F = K1 * (freq)**(5/2) * (1 - np.exp(-tau))
    return F

def F_FFA(freq, K1, K2, alpha):
    """
    Calculate the flux density using the Free-Free Absorption (FFA) model.

    Parameters:
    freq (float or numpy.ndarray): Frequency values.
    K1 (float): Scaling factor.
    K2 (float): Scaling factor for the optical depth.
    alpha (float): Power-law index.

    Returns:
    numpy.ndarray: Flux density values.
    """
    tau = K2 * (freq**(-2.1))
    F = K1 * (freq**(-alpha)) * np.exp(-tau)
    return F

## Fitting to the SSA model

Find the best fit to the data using curve_fit. These will be used as the priors for the MCMC analysis.

In [60]:
def calc_params(data, model):
    # define variables from data
    freq = data['freq']
    flux = data['flux']
    flux_err = data['flux_err']

    params, covariance = curve_fit(model, freq, flux, sigma=flux_err, absolute_sigma=True)

    K1_fit, K2_fit, p_fit = params
    K1_err, K2_err, p_err = np.sqrt(np.diag(covariance))

    # Create a dictionary to store the results
    results = {
        'K1': (K1_fit, K1_err),
        'K2': (K2_fit, K2_err),
        'p': (p_fit, p_err)
    }

    return results

In [61]:
# take the subset of the data at ~1360 days:
data_1 = data[data['time'] < 1370.0]
calc_params(data_1, F_SSA)

RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 800.

Define a prior that imposes the following conditions on our fit. We will input the paramters into the prior as a tuple.
- $K_1$
- $K_2$
- $p$

In [ ]:
# uniform prior:
def lnprior(theta):
    K1, K2, p = theta
    return 0.0

We will now write a likelihood function that takes the SED parameters inside the tuple theta, along with the observed data.

In [ ]:
# F is the flux densities from the data
# F_err are the rms values from the data
def lnlike(theta, nu, F, F_err):
    K1, K2, p = theta

    model = F_SSA(nu, K1, K2, p)
    inv_sigma2 = 1.0/F_err**2

    return -0.5*(np.sum((F-model)**2*inv_sigma2 - np.log(inv_sigma2)))

Now write a function to calculate the marginal probability using the lnlike() and lnprior() functions you calculated above

In [ ]:
def lnprob(theta, nu, F, F_err):
    lp = lnprior(theta)

    if not np.isfinite(lp):
        return -np.inf

    return lp + lnlike(theta, nu, F, F_err)

In [62]:
def get_starting_pos(nwalkers, ndim=3):
    K1 = 100
    K2 = 150
    p = 1
    
    pos = [np.asarray([K1, K2, p]) + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    
    return pos